<a href="https://colab.research.google.com/github/arithama/Organizador-de-Anotacoes-Gemini/blob/main/organizador-de-anotacoes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importando as bibliotecas necessárias e configurando a chave. Coloque a sua própria API key acessando a aba "Secrets" no menu lateral do Google Colab

In [ ]:
!pip install -q google-generativeai
!pip install -q PyDrive

import google.generativeai as genai
import pathlib
import textwrap
import json
import shutil
import os
from google.colab import files
from google.colab import userdata
from google.colab import drive
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from vertexai.generative_models import GenerationConfig
from PIL import Image

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)


Configurando o modelo e pegando o acesso ao Google Drive

In [ ]:
model = genai.GenerativeModel('models/gemini-pro-vision')

config = {
    "max_output_tokens": 2048,
    "temperature": 0.1,
    "top_p": 1,
    "top_k": 16
}

instruction = 'Você analisa fotos de anotações de estudantes. Eles colocam a matéria, a data e o conteúdo no topo da folha. Exemplo: "História - Segunda Guerra Mundial - 08/05/2024". O retorno será apenas a string e os valores não podem ter mais de 3 palavras: {"Matéria": "matéria", "Data": "data", "Conteúdo": "conteúdo"}'

drive.mount('/content/gdrive')
service = build('drive', 'v3')


Mounted at /content/gdrive


Criando o diretório base "Anotações" no seu Google Drive. As pastas das matérias ficarão nele. Se ele já existir, pule esse passo.

In [ ]:
os.makedirs('/content/gdrive/MyDrive/Anotações')

Faça o upload das imagens na janela de input que será aberta em baixo da célula de código. O modelo buscará as palavras chaves no cabeçalho da anotação e as guardará em um objeto. Depois, criará as subpastas de acordo com a matéria, além de modificar o nome do arquivo para ficar mais fácil de buscá-los.


In [ ]:

print("\033[1mBem vindo ao sistema de organização de anotações automático. Faça o upload das suas anotações:\033[0m")
uploaded = files.upload()

if uploaded:
  for file in uploaded:
    image = Image.open(f'/content/{file}')
    response = model.generate_content([f"{instruction}", image], generation_config = config, stream=False)
    response.resolve()
    print(response.text)
    parsed_response = json.loads(response.text)
    print(parsed_response)

    # Accessing the parsed data
    subject = parsed_response["Matéria"]
    date = parsed_response["Data"]
    content = parsed_response["Conteúdo"]

    # Printing the parsed data
    print("Subject:", subject)
    print("Date:", date)
    print("Content:", content)

    folder_path = f'/content/gdrive/My Drive/Anotações/{subject}'
    new_filename = f"{content}-{date.replace('/', '-')}"

    if not os.path.exists(folder_path):
    # If the folder doesn't exist, create it
      os.makedirs(folder_path)
      print(f"Pasta para a nova matéria {subject} criada.")
    else:
      print(f"Pasta para a matéria {subject} já existia.")

    new_image_path = os.path.join(folder_path, f"{new_filename}.jpg")
    shutil.move(f"/content/{file}", new_image_path)
    drive.flush_and_unmount()
    drive.mount('/content/gdrive', force_remount=True)

print("Suas anotações foram organizadas com sucesso, elas já estão no seu Google Drive!")

In [ ]:
drive.flush_and_unmount()